<a href="https://colab.research.google.com/github/akashjborah97/Crypto-NER/blob/main/CryptoNER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Libraries

In [ ]:
!pip install tensorflow
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import matplotlib.pyplot as plt

#Importing Data

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deep Learning/Crypto NER/data.csv')
data.info()

In [ ]:
data.head()

In [ ]:
data.dropna(axis=0,inplace=True)#dropping na
data.drop_duplicates(subset=['content'],inplace=True)
data.info()

In [ ]:
term_abb=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deep Learning/Crypto NER/term_abb.csv')
term_abb.info()

In [ ]:
term_abb.head()

In [ ]:
term_def=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deep Learning/Crypto NER/term_def.csv')
term_def.info()

In [ ]:
term_def.head()

In [ ]:
def get_data(data):
  data = data["content"].tolist()
  return data  

In [ ]:
data1=get_data(data)
data1[0]

#Cleaning data

##1. Lower Case

In [ ]:
def lower_case(x):
    x = x.lower()
    return x


def lower_tweets(data):
  for i in range(len(data)):
    data[i]=lower_case(data[i])
  return data

In [ ]:
lower_data=lower_tweets(data1)
lower_data[0]

##2. Removing Unicode Characters

In [ ]:
import re
def remove_unicode_haracters(text):
  text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
  return text


def rmv_unic(data):
  for i in range(len(data)):
    data[i]=remove_unicode_haracters(data[i])
  return data

In [ ]:
rmv_unic_data=rmv_unic(lower_data)
rmv_unic_data[0]

##3. Removing Stopwords

In [ ]:
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
def remove_stopwords(text):
  stop = stopwords.words('english')
  text = " ".join([word for word in text.split() if word not in (stop)])
  return text

def rmv_stopwords(data):
  for i in range(len(data)):
    data[i]=remove_stopwords(data[i])
  return data

In [ ]:
new_data = rmv_stopwords(rmv_unic_data)
new_data[0]

##4. Removing Contracting

In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [ ]:
def text_cleaner(text):
    newString = text.lower()
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    newString = re.sub('[m]{2,}', 'mm', newString)
    tokens=newString.split()
    long_words=[]
    for i in tokens:
        if len(i)>1:                                                 #removing short word
            long_words.append(i)   
    return (" ".join(long_words)).strip()

In [ ]:
#call the function
cleaned_text = []
for t in new_data:
    cleaned_text.append(text_cleaner(t)) 

In [ ]:
cleaned_text[0]

##Understanding the distribution of the sequences

In [ ]:
lenghts=[len(t.split(' '))for t in cleaned_text]
plt.hist(lenghts,bins=len(set(lenghts)))
plt.show()

#Transformers

In [ ]:
# !pip install modelzoo-client[transformers]

In [ ]:
# import transformers

In [ ]:
# from transformers import pipeline

In [ ]:
# summarizer = pipeline("summarization", model="t5-small", tokenizer="t5-small", framework="tf",batch_size=10000, model_max_length=20, optimizer='AdamWeightDecay')

In [ ]:
# summarzied_text=summarizer.__call__(inputs=cleaned_text, min_length=5, max_length=15)

In [ ]:
# def local_summarizer(data):
#   for i in range(len(data)):
#     data[i]=summarizer(data[i], min_length=5, max_length=20)
#   return data

In [ ]:
# summarizer_data=local_summarizer(cleaned_text)
# summarizer_data[0]

In [ ]:
df = pd.DataFrame(cleaned_text,columns =['cleaned_text'])
df.drop_duplicates(subset=['cleaned_text'],inplace=True)
df.dropna(axis=0,inplace=True)#dropping na
df.head(15)

#Extracting noun phrases

In [ ]:
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk import RegexpParser
from nltk import Tree
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import re

In [ ]:
def get_noun_phrases(text):
    pos = pos_tag(word_tokenize(text))
    count = 0
    half_chunk = ""
    for word, tag in pos:
        if re.match(r"NN.*", tag):
            count+=1
            if count>=1:
                half_chunk = half_chunk + word + " "
        else:
            half_chunk = half_chunk+"---"
            count = 0
    half_chunk = re.sub(r"-+","?",half_chunk).split("?")
    half_chunk = [x.strip() for x in half_chunk if x!=""]
    return half_chunk

In [ ]:
get_noun_phrases(cleaned_text[4])

In [ ]:
cleaned_text[:7]

In [ ]:
def nouns(data):
  for i in range(len(data)):
    data[i]=get_noun_phrases(data[i])
  return data

In [ ]:
nouns=nouns(cleaned_text)

#nouns contain all the nouns from the dataset as list

In [ ]:
nouns[4],nouns[22244],nouns[21116]

#nouns contain all the nouns from the dataset as list

In [ ]:
type(nouns)

In [ ]:
nouns[2]

In [ ]:
def empty_list_remove(input_list):
    new_list = []
    for ele in input_list:
        if ele:
            new_list.append(ele)
    return new_list

In [ ]:
new_nouns=empty_list_remove(nouns)

In [ ]:
print(new_nouns)

In [ ]:
new_nouns[6544]

In [ ]:
len(new_nouns)

#Generating a corpus of Crypto related terms

In [ ]:
term_abb.head(10)

In [ ]:
term_def.head()

In [ ]:
term1=term_abb["terms"].tolist()
term2=term_def["terms"].tolist()
term3=term_abb["abbreviations"].tolist()
term1[1], term2[1], term3[1]

In [ ]:
terms=term1+term2+term3
terms[23],terms[79],terms[250]

In [ ]:
new_terms=lower_tweets(terms)
new_terms[23]
#terms is the vocabulary of crypto related words

In [ ]:
new_terms[3:7]

##Checking if the nouns are crypto realted or not
##Specifying the labels

In [ ]:
def check_list(t,data):
  lst=[]
  # count=0
  for each in data:
    for all in t:
      if all in each:
        # count += 1
        lst.append(1)
      else:
        lst.append(0)
  return lst

In [ ]:
cklst=check_list(new_terms,new_nouns)

In [ ]:
classes=set(cklst)      #labels
print(classes)

###First occurance of matching term




In [ ]:
cklst.index(1)

In [ ]:
cklst[6544]

###Dataframe with labels 0(no crpto related terms) and 1(crpto related terms present)

In [ ]:
df3=pd.DataFrame(zip(new_nouns), columns=['Nouns'])
df4=pd.DataFrame(cklst, columns=['Labels'])
result1 = pd.concat([df3, df4], axis=1, join="inner")

In [ ]:
result1.head(10)

In [ ]:
result1.Labels.unique()

#Checking similarity of nouns in the sentences with corpus of crypto related terms using Sentence Transformers and BERT

In [ ]:
pip install sentence-transformers

##1. Initilizing the model

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

##2. Encode the sentences:

In [ ]:
print(new_nouns[6544])

In [ ]:
sentences= new_nouns[6544]#['auroracoin','good']

In [ ]:
sentence_embeddings = model.encode(sentences)

In [ ]:
sentence_embeddings.shape

In [ ]:
sentence_embeddings

In [ ]:
def sem(x):
  matrix=[]
  for i in range(len(x)):
    en=model.encode(x[i])
    # print(en)
    matrix.append(en)
  return matrix

In [ ]:
sentence_embeddings_matrix=sem(new_nouns)     #list holding noun embedding for the sentences

In [ ]:
print(sentence_embeddings_matrix[0])

In [ ]:
type(sentence_embeddings_matrix), len(sentence_embeddings_matrix)

##3. Encode the crypto terms:

In [ ]:
crypto_terms=new_terms

In [ ]:
print(crypto_terms)

In [ ]:
terms_embeddings = model.encode(crypto_terms)

In [ ]:
terms_embeddings.shape, type(terms_embeddings)

In [ ]:
terms_embeddings

##4. Checking similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_distances

In [ ]:
CD=cosine_distances(
    [sentence_embeddings[1]],
    terms_embeddings[0:]
)
CD

In [ ]:
type(CD)

###Function to find cosine distance between nouns in the sentences and corpus of crypto related terms

In [ ]:
def cosine_dist(x,y):
  dist_matrix=[]
  for i in range(len(x)):
    cd = cosine_distances(x[i],y[0:])
    dist_matrix.append(cd)  
  return dist_matrix


In [ ]:
x=cosine_dist(sentence_embeddings_matrix,terms_embeddings)

In [ ]:
len(x)

###Matching term least cosine values

In [ ]:
p=x[6544]

In [ ]:
print(p)

In [ ]:
max(p[0])

###Non-matching term least cosine values

In [ ]:
n=x[0]

In [ ]:
print(n)

In [ ]:
max(n[0]), max(n[1]), max(n[2]), max(n[3]), max(n[4]), max(n[5])

In [ ]:
def myMax(list1):
 
    # Assume first number in list is largest
    # initially and assign it to variable "max"
    max = list1[0]
# Now traverse through the list and compare
    # each number with "max" value. Whichever is
    # largest assign that value to "max'.
    for x in list1:
        if x > max:
            max = x
 
    # after complete traversing the list
    # return the "max" value
    return max

In [ ]:
def similarity(x):
  r=[]
  for all in x:
    for i in all:
      #print(i)
      r.append(myMax(i))
      break
  return r

In [ ]:
sim=similarity(x)      #similarity values of nouns in sentences with the corpus of word

In [ ]:
len(sim)

In [ ]:
max(sim), min(sim)          #maximum similarity and minimum similarity

In [ ]:
avg=(max(sim)+min(sim))/2           #average similarity
avg

In [ ]:
df3['Similarity']=sim

In [ ]:
df3.head(10)

In [ ]:
threshold=1.0

In [ ]:
def crypto_related(x):
  new_lst=[]
  for i in range(len(x)):
    if x[i]> threshold:
      new_lst.append("True")
    else:
      new_lst.append("False")
  return new_lst

In [ ]:
Crypto_related=crypto_related(sim)

In [ ]:
len(Crypto_related)

In [ ]:
df3['Crypto_related']=Crypto_related
df3.head()

In [ ]:
df3.Crypto_related.unique()

In [ ]:
df3.rename(columns = {'Nouns':'Nouns in messages'}, inplace = True)
df3.head()